<a href="https://colab.research.google.com/github/MatheusHiro/Portfolio/blob/main/multi_subject_dreambooth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Carrega a biblioteca/Clona o repositório ⬇**

In [ ]:
!git clone https://github.com/huggingface/diffusers


In [ ]:
cd diffusers

In [ ]:
!pip install -e .


In [ ]:
cd examples/research_projects/multi_subject_dreambooth

In [ ]:
!pip install -r requirements.txt

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#@title Login to HuggingFace 🤗

#@markdown You need to accept the model license before downloading or using the Stable Diffusion weights. Please, visit the [model card](https://huggingface.co/runwayml/stable-diffusion-v1-5), read the license and tick the checkbox if you agree. You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work.
# https://huggingface.co/settings/tokens
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = "" #@param {type:"string"}
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

In [ ]:
#@markdown Name/Path of the initial model.
MODEL_NAME = "runwayml/stable-diffusion-v1-5" #@param {type:"string"}



#@markdown Enter the directory name to save model at.

OUTPUT_DIR = "" #@param {type:"string"}
OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR


print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $OUTPUT_DIR


In [ ]:
!pip install xformers
!pip install bitsandbytes

In [ ]:
!pip3 install datasets

**Treinamento do modelo ⬇**

In [ ]:
!python3 train_multi_subject_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --concepts_list="" \
  --output_dir=$OUTPUT_DIR \
  --mixed_precision="fp16" \
  --prior_loss_weight=1.0 \
  --num_class_images=0 \
  --resolution=512 \
  --gradient_accumulation_steps=1 --gradient_checkpointing \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=20000 \
  --checkpointing_steps=15000


**Geração das Imagens ⬇**

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image as PILImage
import os
import datetime

model_path = ""

# Create the output folder if it doesn't exist
output_folder = ""
os.makedirs(output_folder, exist_ok=True)

pipe = StableDiffusionPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16).to("cuda")
pipe.to("cuda")

prompt = ""

# Save 10 images
images = pipe(prompt, num_images_per_prompt=10).images

for i, image in enumerate(images):
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")  # Add timestamp

    if "t@y" in prompt:
        folder_name = "CEC"
        output_filename = f"CEC_{timestamp}_{i}.png"
    elif "sks" in prompt:
        folder_name = "Controle"
        output_filename = f"Controle_{timestamp}_{i}.png"
    elif "niu" in prompt:
        folder_name = "Exposed"
        output_filename = f"Exposed_{timestamp}_{i}.png"
    elif "oxi" in prompt:
        folder_name = "LPM"
        output_filename = f"LPM_{timestamp}_{i}.png"
    else:
        folder_name = "Other"
        output_filename = f"Cell_{timestamp}_{i}.png"

    # Save the image to the specified folder
    output_path = os.path.join(output_folder, output_filename)
    image.save(output_path)

    # Save the text to a txt file


    print(f"Image {i+1} saved to: {output_path}")

